In [1]:
!pip install pyspark

In [ ]:
# for databricks

# %sh
# git clone https://github.com/VincentHenric/IASD_privacy.git
    
# import sys
# sys.path.append('/databricks/driver/IASD_privacy')

In [2]:
# Mount drive
import os
from google.colab import drive
%cd /content
drive.mount('/content/drive', force_remount=True)
%cd 'drive/My Drive/Colab Notebooks/privacy'
%ls -l
print(os.listdir())

java8_location= '/usr/lib/jvm/java-8-openjdk-amd64' # Set your own
os.environ['JAVA_HOME'] = java8_location

/content
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/privacy
total 3735352
-rw------- 1 root root        450 Feb 21 18:43  clean_data.py
-rw------- 1 root root      16495 Mar  7 21:26  exp_0.txt
-rw------- 1 root root      23321 Mar  8 01:26  exp_1.txt
-rw------- 1 root root      17778 Mar  8 17:12  exp_3.txt
-rw------- 1 root root      15170 Mar  8 19:23  exp_4.txt
-rw------- 1 root root          0 Mar  9 00:31  exp_5.txt
-rw------- 1 root root       3061 Mar 14 12:09 'experiment (1).py'
-rw------- 1 root root       3061 Mar 14 12:09  experiment.py
drwx------ 2 root root       4096 Feb 21 17:32  netflix-prize-data/
-rw------- 1 root root      72265 Feb 23 11:23  privacy_project.ipynb
-rw------- 1 root root      46270 Mar 14 14:31  privacy_project_spark.ipynb
-rw------- 1 root root      14439 Mar 14 10:24  privacy.py
-rw------- 1 root root       9522 Mar 14 14:02  privacy_spark.py
drwx------ 2 root root       4096 Mar 14 14:05  __pycache__/
-rw------- 1 root root 

In [0]:
from pyspark.sql import *
from pyspark.conf import SparkConf
import pyspark.sql.functions as F
from pyspark.sql import Window
%load_ext autoreload
%autoreload 1
import matplotlib.pyplot as plt
%aimport experiment
%aimport privacy_spark

In [0]:
from config import SPARK_CONF

spark = SparkSession \
    .builder \
    .appName("Privacy Project") \
    .config(conf=SPARK_CONF) \
    .getOrCreate()

print(spark.sparkContext.getConf().getAll())

In [0]:
exp = experiment.Experiment(spark)
exp.load_dataset("./datasets/netflix.ratings.csv")
# exp.load_dataset("FileStore/tables/ratings_*.csv") # for databricks

window = Window.partitionBy('movieId')
exp.df = exp.df.withColumn('avgMovieRating',F.avg('rating').over(window))
exp.df = exp.df.withColumn('nbReviews', F.count('rating').over(window))
exp.df = exp.df.repartition('custId').cache()
exp.df = exp.df.repartition('custId').cache()

### Experiment with Movie

In [9]:
no_info = privacy_spark.Auxiliary(False, False, 0, 0)
ratings_exact_date_3  = privacy_spark.Auxiliary(True, True, 0, 3)
ratings_exact_date_14 = privacy_spark.Auxiliary(True, True, 0, 14)

aux_list = 3*[ratings_exact_date_14] + 1*[no_info]
print(exp.evaluate(aux_list, N=100))

60.0


In [0]:
scoring = exp.get_scoring(similarity='netflix', with_movie=True)
auxiliary = exp.generate_auxiliary_data(aux_list, N=100)
scores = scoring.compute_score(exp.spark.createDataFrame(auxiliary), exp.df)
matching_set = scoring.matching_set(scores, 0).toPandas()
# scoring.output(scores, 0.5)

NameError: name 'scoring' is not defined

### Experiment without Movie

In [0]:
no_info = privacy_spark.Auxiliary(False, False, 0, 0)
ratings_exact_date_3  = privacy_spark.Auxiliary(True, True, 0, 3)
ratings_exact_date_14 = privacy_spark.Auxiliary(True, True, 0, 14)
ratings_exact_date_0  = privacy_spark.Auxiliary(True, True, 0, 0)

aux_list = 6*[ratings_exact_date_3]# + 2*[no_info]

In [ ]:
scoring = exp.get_scoring(similarity='netflix', with_movie=False)
auxiliary = exp.generate_auxiliary_data(aux_list, N=10)
scores = scoring.compute_score(exp.spark.createDataFrame(auxiliary), exp.df)
matching_set = scoring.matching_set(scores, 0).toPandas()

In [ ]:
matching_set

In [0]:
scoring = exp.get_scoring(similarity='netflix', with_movie=False)
scores_individual = scoring.compute_individual_score(exp.spark.createDataFrame(auxiliary), exp.df)

In [27]:
# aux = exp.generate_auxiliary_data(aux_list, 1)
# scores = exp.compute_score(aux, similarity='netflix', with_movie=False).sort('value', ascending=False)
# scores.show()
exp.evaluate(aux_list, N=100, similarity='netflix', with_movie=False)

98.0